# Noiseless VQE Simulation of Lithium Hydride (LiH)

This notebook implements the **Variational Quantum Eigensolver (VQE)** algorithm to estimate the **ground state energy of lithium hydride (LiH)** using PennyLane’s quantum chemistry module. The simulation is performed on a **noiseless quantum device**, focusing exclusively on **double excitations** within a Unitary Coupled Cluster (UCC) ansatz.

### Key Features:
- **Molecular Hamiltonian Generation**: Constructs the LiH Hamiltonian using the STO-3G basis and maps it to a qubit representation.
- **Reference State**: Prepares the Hartree-Fock (HF) state from the mean-field solution.
- **Ansatz Construction**: Applies a double excitation ansatz derived from the UCCSD method.
- **VQE Optimization**: Minimizes the expectation value of the Hamiltonian using **Gradient Descent**.
- **Results Analysis**:
  - Convergence plot of energy vs. iteration.
  - Visualization of the ground state quantum amplitudes.

### Purpose:
To benchmark and analyze the noiseless behavior of VQE on LiH as a testbed for quantum chemistry applications. This forms a foundation for:
- Testing optimizers,
- Comparing excitation strategies,

In [ ]:
import pennylane as qml                  # Quantum circuit builder and device management
from pennylane import numpy as np        # Not regular NumPy, but similar and supports automatic differentiation
from pennylane import qchem              # Quantum chemistry module used to define molecule Hamiltonians
from pennylane.qchem import excitations  # Single and double excitations used in the UCCSD (Unitary Coupled Cluster Singles and Doubles) ansatz
import matplotlib.pyplot as plt
import json, os, time
import hashlib, glob
from vqe_utils import excitation_ansatz, get_optimizer, set_seed, make_run_config_dict, run_signature, find_existing_run, build_run_filename

seed = 0
set_seed(seed)                           # Reproducible runs
IMG_DIR = "images"                       # Single image directory used repo-wide
os.makedirs(IMG_DIR, exist_ok=True)
RUNS_DIR = "runs"                        # Run record directory used repo-wide
os.makedirs(RUNS_DIR, exist_ok=True)

In [ ]:
# Define the atoms in the LiH molecule
symbols = ["Li", "H"]

# Define the coordinates (in Ångströms)
coordinates = np.array([
    [0.0, 0.0, 0.0],  # Lithium atom at the origin
    [0.0, 0.0, 1.6]   # Hydrogen atom positioned 1.6 Å along the z-axis
])

# Define the LiH Hamiltonian and the number of qubits required
# Default STO-3G basis set
basis = "STO-3G"
hamiltonian, qubits = qchem.molecular_hamiltonian(symbols, coordinates, charge=0, basis=basis)

# 1 spatial orbital for H and 5 for Li
# Each spatial orbital is doubly-degenerate (2 spin orbitals)
print(f"{qubits} qubits required")

In [ ]:
# Which orbitals are occupied (1) or unoccupied (0) in the mean-field (Hartree-Fock) solution
electrons = 4  # 1 from H and 3 from Li
hf = qchem.hf_state(electrons=electrons, orbitals=qubits)  # Creates the Hartree-Fock state

# Show that the first 4 orbitals are occupied
# Print the state in bra-ket notation (leftmost bit = highest-index spin orbital)
print(f"Hartree-Fock state: {'|' + ''.join(str(bit) for bit in hf) + '⟩'}")

In [ ]:
# Define the number of required quantum wires / logical qubits
num_wires = qubits

# Create quantum device simulator backend
dev = qml.device("default.qubit",  # Noiseless state vector simulator
                 wires=num_wires)


@qml.qnode(dev)  # Transforms exp_energy below into a quantum node
def exp_energy(state):
    qml.BasisState(np.array(state), wires=range(num_wires))

    # Return the expectation value of the molecular Hamiltonian
    return qml.expval(hamiltonian)

# Calculate ⟨ψ_hf| hamiltonian |ψ_hf⟩ in Hartree (Ha) units
# 1 Ha = 27.2 eV
exp_energy(hf)

In [ ]:
# Generate excitation indices
singles, doubles = excitations(electrons=electrons, orbitals=qubits)
print(f"Singles Excitations: {singles}")  # e.g: [0, 4] is a single excitation of an electron in orbital 0 to orbital 4
print(f"Doubles Excitations: {doubles}")  # e.g: [0, 1, 4, 5] is a double excitation of electrons in orbitals 0 and 1 to orbitals 4 and 5

In [ ]:
ansatz_desc = "UCC doubles (DoubleExcitation over doubles)"

# Define the VQE cost function
@qml.qnode(dev)
def cost_function(params):
    excitation_ansatz(params, wires=range(num_wires), hf_state=hf, excitations=doubles, excitation_type="double")

    # Measure the expectation value of the Hamiltonian after applying the ansatz:
    # E(theta) = ⟨ψ(theta)| H |ψ(theta)⟩
    return qml.expval(hamiltonian)

# Create a vector of zeros with the same length as the number of double excitations
initial_params = np.zeros(len(doubles), requires_grad=True)

# Confirm the initial energy of the system using the Hartree-Fock state
# This is the starting point for classical optimization
cost_function(initial_params)

In [ ]:
# The optimizer uses automatic differentiation to compute gradients and adjust the parameters
stepsize = 0.2
max_iterations = 50
optimizer = "GradientDescent"
opt = get_optimizer(optimizer, stepsize=stepsize)

# Build configuration and signature
cfg = make_run_config_dict(
    symbols=symbols,
    coordinates=coordinates,
    basis=basis,
    ansatz_desc=ansatz_desc,
    optimizer_name=optimizer,
    stepsize=stepsize,
    max_iterations=max_iterations,
    seed=seed,
)
sig = run_signature(cfg)
existing = find_existing_run(RUNS_DIR, sig)

if existing:
    # Load results if this configuration has been run before
    with open(existing) as f:
        rec = json.load(f)
    print(f"[reuse] Loaded existing run: {existing}")
    energy = rec["energies"]

    # Accept both old "final_params" (list) or "params_by_step" (last entry)
    if "final_params" in rec:
        theta = np.array(rec["final_params"], requires_grad=True)
    elif "params_by_step" in rec and rec["params_by_step"]:
        theta = np.array(rec["params_by_step"][-1], requires_grad=True)
    else:
        # Fallback if absent — compute from scratch
        existing = None
        print("[reuse] No parameters stored; recomputing.")
else:
    print("[compute] No matching run found; optimizing now.")

if not existing:
    # Compute results if this configuration has NOT been ran before
    theta = np.zeros(len(doubles), requires_grad=True)
    energy = [cost_function(theta)]

    # Optimization loop
    for _ in range(max_iterations):
        theta, prev_energy = opt.step_and_cost(cost_function, theta)
        energy.append(prev_energy)
        
    # Update the final point:
    energy[-1] = cost_function(theta)

    run_record = {
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),  # Date and time of run
        "molecule": symbols,
        "geometry": coordinates.tolist(),
        "basis": basis,
        "electrons": electrons,
        "num_wires": num_wires,
        "ansatz": ansatz_desc,
        "optimizer": {"name": optimizer, "stepsize": stepsize, "iterations_planned": max_iterations, "iterations_ran": len(energy)-1},
        "seed": seed,
        "energies": [float(e) for e in energy],     # List of energies per iteration
        "final_params": [float(x) for x in theta],  # Store only final params
        "config_hash": sig,
    }

    # Canonical file name: readable prefix + signature (no timestamp needed)
    fname = f"{RUNS_DIR}/LiH_Noiseless_{optimizer}_s{seed}__{sig}.json"
    with open(fname, "w") as f:
        json.dump(run_record, f, indent=2)
    print("Saved run:", fname)

# Plotting VQE convergence
plt.plot(range(len(energy)), energy)
plt.xlabel('Iteration')
plt.ylabel('Energy (Ha)')
plt.title('LiH VQE Convergence (Noiseless, UCC Doubles)')
plt.grid(True)
plt.tight_layout()
plt.savefig(f'{IMG_DIR}/LiH_Gradient_Descent.png', dpi=300)
plt.show()

In [ ]:
print(f"Final ground state energy: {energy[-1]:.8f} Ha")

# Optimized angles in the DoubleExcitation gates for first two excitations
print(f"Final parameters: {theta[0]:.8f}, {theta[1]:.8f}")

In [ ]:
@qml.qnode(dev)
def ground_state(params):
    excitation_ansatz(
        params,
        wires=range(num_wires),
        hf_state=hf,
        excitations=doubles,
        excitation_type="double",
    )

    # Return the entire quantum statevector
    return qml.state()

# Run the circuit using the final optimized angles
final_state = ground_state(theta)

# Find non-zero (or near non-zero) entries
threshold = 1e-2  # Recommended smaller threshold to capture all significant amplitudes
non_zero_indices = np.where(np.abs(final_state) > threshold)[0]
non_zero_amplitudes = final_state[non_zero_indices]

# Build the full ket notation string
ket_terms = []
for idx, amp in zip(non_zero_indices, non_zero_amplitudes):
    binary_state = f"|{idx:0{num_wires}b}⟩"
    amp_str = f"{amp.real:.4f}" if np.isclose(amp.imag, 0, atol=1e-6) else f"({amp.real:.4f} + {amp.imag:.4f}j)"
    ket_terms.append(f"{amp_str}{binary_state}")

# Join all terms into one quantum state expression and print
ket_notation = " + ".join(ket_terms)
print(f"Ground state of LiH:\n|ψ⟩ = {ket_notation}")

In [ ]:
# Prepare labels and amplitudes for the plot
labels = [f"|{i}⟩" for i in non_zero_indices]  # decimal labels
amplitudes = np.abs(non_zero_amplitudes)

plt.figure(figsize=(12, 6))
plt.bar(labels, amplitudes)
plt.xlabel('Basis States (decimal)', fontsize=14)
plt.ylabel('Amplitude', fontsize=14)
plt.title('Ground State of LiH (VQE Result)', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig(f'{IMG_DIR}/LiH_Ground_State.png', dpi=300)
plt.show()